In [325]:
import logging
import os
import openai
import azure.functions as func
from azure.identity import DefaultAzureCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import QueryType
from azure.core.credentials import AzureKeyCredential
import json
from azure.cosmos import CosmosClient
import hashlib
from azure.keyvault.secrets import SecretClient

In [326]:
# Replace these values with your Azure Key Vault URL
keyvault_url = "https://kv-nie.vault.azure.net/"

# Create a SecretClient using the DefaultAzureCredential
credential = DefaultAzureCredential()
client_kv = SecretClient(vault_url=keyvault_url, credential=credential)

In [327]:
# Replace these with your own values, either in environment variables or directly here
AZURE_STORAGE_ACCOUNT = client_kv.get_secret("azure-storage-account").value
AZURE_SEARCH_SERVICE = client_kv.get_secret("azure-search-service").value
AZURE_SEARCH_INDEX = client_kv.get_secret("azure-search-index").value
AZURE_OPENAI_SERVICE = client_kv.get_secret("azure-openai-service").value
AZURE_OPENAI_CHATGPT_DEPLOYMENT = client_kv.get_secret("azure-openai-chatgpt").value
AZURE_OPENAI_EMB_DEPLOYMENT = os.environ.get("AZURE_OPENAI_EMB_DEPLOYMENT") or "embedding"
AZURE_SEARCH_API_KEY = client_kv.get_secret("azure-search-api-key").value
AZURE_OPENAI_GPT_DEPLOYMENT = os.environ.get("AZURE_OPENAI_GPT_DEPLOYMENT") or "davinci"

KB_FIELDS_CONTENT = os.environ.get("KB_FIELDS_CONTENT") or "content"
KB_FIELDS_CATEGORY = os.environ.get("KB_FIELDS_CATEGORY") or "category"
KB_FIELDS_SOURCEPAGE = os.environ.get("KB_FIELDS_SOURCEPAGE") or "sourcepage"

In [328]:
credential = AzureKeyCredential(AZURE_SEARCH_API_KEY)

# Used by the OpenAI SDK
openai.api_base = f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com"
openai.api_version = "2023-05-15"

# Comment these two lines out if using keys, set your API key in the OPENAI_API_KEY environment variable and set openai.api_type = "azure" instead
openai.api_type = "azure"
openai.api_key = "4657af893faf48e5bd81208d9f87f271"

# Set up clients for Cognitive Search and Storage
search_client = SearchClient(
    endpoint=f"https://{AZURE_SEARCH_SERVICE}.search.windows.net",
    index_name=AZURE_SEARCH_INDEX,
    credential=credential)

In [329]:
base_system_message = """You are a faculty who assists teachers design a course outline for their students. 
Answer ONLY with the facts listed in the list of sources below. If there isn't enough information below, say you don't know. 
Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.
"""

system_message = f"{base_system_message.strip()}"

In [330]:
import tiktoken

def num_tokens_from_messages(messages, model="gpt-4-32k"):
    encoding = tiktoken.encoding_for_model(model)
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # if there's a name, the role is omitted
                num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2  # every reply is primed with <im_start>assistant
    return num_tokens

In [331]:
# Defining a function to send the prompt to the ChatGPT model
# More info : https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions
def send_message(messages, model_name, max_response_tokens=500):
    response = openai.ChatCompletion.create(
        engine=AZURE_OPENAI_CHATGPT_DEPLOYMENT,
        messages=messages,
        temperature=0.5,
        max_tokens=max_response_tokens,
        top_p=0.9,
        frequency_penalty=0,
        presence_penalty=0,
    )
    return response['choices'][0]['message']['content']

# Defining a function to print out the conversation in a readable format
def print_conversation(messages):
    for message in messages:
        print(f"[{message['role'].upper()}]")
        print(message['content'])
        print()

In [332]:

# This is the first user message that will be sent to the model. Feel free to update this.
engage = """
        Please help to Explain constructive alignment. Describe and explain what is a blended course Describe and explain what are modes of learning (face to face, synchronous and/or asynchronous). 
        Explain Intended Learning Outcomes(ILO) that uses the ABCD (Audience, Behaviour, Condition, Degree) model in relation to the Bloom’s Taxonomy for the ‘behaviour’ component. 
        Describe and explain the verbs used for "Behaviour" component. Provide an example and identify which are the ABCD components. 
        Explain how more than one ILO can be achieved in one lesson in a course. 
        List a variety of both online and face to face learner – centred Teaching and Learning Activities (TLA). List a variety of corresponding Assessment Tasks (AT) which are either formative or summative. 
        Describe what it means to weave the TLAs from lesson to lesson. 
        List a variety of ed-tech (e.g. padlet, kahoot) tools to complement 3 modes of learner-centred TLAs. List a variety of other resources such as articles, books, videos to complement 3 TLAs. 
        Explain what should be included in writing a course synopsis.
        Explain and analyze what is TE21 Framework and Components of V3SK . 
        Be detailed in your answers. Answer by adapting the frameworks listed in the list of sources below.
        """

In [333]:
exclude_category = None
query_vector = openai.Embedding.create(engine=AZURE_OPENAI_EMB_DEPLOYMENT, input=engage)["data"][0]["embedding"]

# Alternatively simply use search_client.search(q, top=3) if not using semantic search
filter = "category ne '{}'".format(exclude_category.replace("'", "''")) if exclude_category else None
r = search_client.search(
                        engage, 
                        filter=filter,
                        query_type=QueryType.SEMANTIC, 
                        query_language="en-us", 
                        query_speller="lexicon", 
                        semantic_configuration_name="default", 
                        # top=3,
                        vector=query_vector if query_vector else None, 
                        top_k=50 if query_vector else None,
                        vector_fields="embedding" if query_vector else None
                        )
results = results = [str(doc[KB_FIELDS_SOURCEPAGE]) + ": " + str(doc[KB_FIELDS_CONTENT]).replace("\n", "").replace("\r", "") for doc in r]
content = "\n".join(results)

In [334]:

# This is the first user message that will be sent to the model. Feel free to update this.
user_message = engage + " \nSOURCES:\n" + content

In [335]:
# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

In [336]:

token_count = num_tokens_from_messages(messages)
print(f"Token count: {token_count}")

Token count: 156180


In [337]:

max_response_tokens = 2048

response = send_message(messages, AZURE_OPENAI_CHATGPT_DEPLOYMENT, max_response_tokens)
messages.append({"role": "assistant", "content": response})

InvalidRequestError: This model's maximum context length is 16384 tokens. However, your messages resulted in 156181 tokens. Please reduce the length of the messages.

In [ ]:
Input = "a 5-week course outline on Food Fiction: A Creative Writing and Storytelling"
explore = """
            Thanks for your response, then with the the given explanations, create “{Input}” provide a course synopsis for the entire course within 200 words.
            Write a maximum of 5 ILOs,using the ABCD model, for the entire 
            course.
            For each week, list the topic and mode of delivery determined by 
            the nature of the content and TLAs. 
            For each week, provide a variety of TLAs and a variety of 
            formative assessment tasks (ATs) for the first 4 weeks and a 
            summative assessment task for the last week. Ensure the ATs 
            and TLAs are weaved.
            For each week, alllow for more than one ILO to be achieved and 
            state which ILOs has been achieved in each lesson.
            Suggest ed tech tools to support the TLAs in each week.
            Suggest other learning resources (books,websites,journals) to 
            support the TLAs in each week".
            """.format(Input = Input)
#user_message = "Let's talk about generative AI and keep the tone informational but also friendly."
#user_message = "Show me a few more examples"
messages.append({"role": "user", "content": explore})

max_response_tokens=2048

response = send_message(messages, AZURE_OPENAI_CHATGPT_DEPLOYMENT, max_response_tokens)

messages.append({"role": "assistant", "content": response})

In [ ]:
explain = """
            Greate! and for the course outline you just created, Examine if the ATs and TLAs support 
            the constructive alignment with the 
            ILOs.
            Explain how are the various ILOs 
            achieved for each lesson
            Examine if the ATs and TLAs from 
            week to week are weaved.
            Explain how the nature of the content 
            and TLAs determine the mode of 
            delivery for each lesson. 
            Making sure, you provide your 
            explanation with reference to the 
            above course outline."""
#user_message = "Let's talk about generative AI and keep the tone informational but also friendly."
#user_message = "Show me a few more examples"
messages.append({"role": "user", "content": explain})

max_response_tokens=2048

response = send_message(messages, AZURE_OPENAI_CHATGPT_DEPLOYMENT, max_response_tokens)

messages.append({"role": "assistant", "content": response})

In [ ]:
elaborate = """
            Excellent! based on the above explaintions and the course you just created, present the 
            following information using a json format with this structure
            {{
                "Course synopsis": Information of Course synopsis,
                "ILOs for entire course": List out maximum of 5 ILOs using the ABCD model, starting with Student will be able to... for each point,
                "TE21": Analyze the keywords from Course synopsis and ILOs then generate TE21 information with skills, values and knowledge applying the components of V3SK
                "Week": [
                    "Week": number of this week,
                    "Course topics": Information of Course topics in this week,
                    "ILOs achieved": Which point number of ILOs achieved in this week,
                    "ATs": Information of ATs in this week,
                    "TLAs": Information of TLAs in this week, The TLAs must be explained clearly, exhibiting weaving from lesson to lesson
                    "Ed tech tools" : Information of Ed tech tools in this week,
                    "Other learning resources" : Information of learning resources in this week
                    "Mode of learning": Information of Mode of learning in this week, 
                        Do take note that 30-60 percent of the course should be online and the nature of the lesson should decide the mode of learning
                        just list out the suitable Mode of learning without percentage of each
                    ]      
            }} 
            """.format(Input = Input)
#user_message = "Let's talk about generative AI and keep the tone informational but also friendly."
#user_message = "Show me a few more examples"
messages.append({"role": "user", "content": elaborate})

max_response_tokens=2048

response = send_message(messages, AZURE_OPENAI_CHATGPT_DEPLOYMENT, max_response_tokens)

messages.append({"role": "assistant", "content": response})

In [ ]:
last_reponse = len(messages)
for i in range(0, len(messages) + 1):
    if i == last_reponse:     
        print(messages[i-1]['content'])